In [1]:
import pandas as pd
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [23]:
# weather_df = pd.read_csv("C:\Github\Group3_Project4\pete files\state weather csv\combined.csv")
# weather_df['Time'] = pd.to_datetime(weather_df['Time'])
# weather_df = weather_df.sort_values(by = 'State', ascending= True).set_index('State')
# weather_df


In [24]:
# weather_df.dtypes
# weather_df['Time'] = pd.to_datetime(weather_df['Time'])


In [6]:
# weather_df.to_csv('5yr_weather_data_US.csv')

In [12]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": [32.806671, 61.370716, 33.729759, 34.969704, 36.116203, 39.059811, 41.597782, 39.318523, 38.897438,
                 27.766279, 33.040619, 21.094318, 44.240459, 40.349457, 39.849426, 42.011539, 38.5266, 37.66814,
                 31.169546, 44.693947, 39.063946, 42.230171, 43.326618, 45.694454, 32.741646, 38.456085, 46.921925,
                 41.12537, 38.313515, 43.452492, 40.298904, 34.840515, 42.165726, 35.630066, 47.528912, 40.388783,
                 35.565342, 44.572021, 40.590752, 41.680893, 33.856892, 44.299782, 35.747845, 31.054487, 40.150032,
                 44.045876, 37.769337, 47.400902, 38.491226, 44.268543, 42.755966],
    "longitude": [-86.79113, -152.404419, -111.431221, -92.373123, -119.681564, -105.311104, -72.755371, -75.507141,
                  -77.026817, -81.686783, -83.643074, -157.498337, -114.478828, -88.986137, -86.258278, -93.210526,
                  -96.726486, -84.670067, -91.867805, -69.381927, -76.802101, -71.530106, -84.536095, -93.900192,
                  -89.678696, -92.288368, -110.454353, -98.268082, -117.055374, -71.563896, -74.521011, -106.248482,
                  -74.948051, -79.806419, -99.784012, -82.764915, -96.928917, -122.070938, -77.209755, -71.51178,
                  -80.945007, -99.438828, -86.692345, -97.563461, -111.862434, -72.710686, -78.169968, -121.490494,
                  -80.954453, -89.616508, -107.30249],
    "start_date": "2000-01-31",
    "end_date": "2023-12-31",
    "hourly": "temperature_2m",
    "daily": ["temperature_2m_max", "temperature_2m_min"],
    "temperature_unit": "fahrenheit",
    "wind_speed_unit": "mph",
    "precipitation_unit": "inch"
}
responses = openmeteo.weather_api(url, params=params)

# Initialize lists to hold hourly and daily data
hourly_dataframes = []
daily_dataframes = []

# Loop through all responses to process data for each location
for response in responses:
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "latitude": response.Latitude(),
        "longitude": response.Longitude(),
        "temperature_2m": hourly_temperature_2m
    }
    hourly_dataframes.append(pd.DataFrame(data=hourly_data))

    daily = response.Daily()
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
    daily_data = {
        "date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left"
        ),
        "latitude": response.Latitude(),
        "longitude": response.Longitude(),
        "temperature_2m_max": daily_temperature_2m_max,
        "temperature_2m_min": daily_temperature_2m_min
    }
    daily_dataframes.append(pd.DataFrame(data=daily_data))

# Concatenate hourly and daily dataframes for all locations
hourly_dataframe = pd.concat(hourly_dataframes, ignore_index=True)
daily_dataframe = pd.concat(daily_dataframes, ignore_index=True)

# print(hourly_dataframe)
# print(daily_dataframe)


In [11]:
daily_dataframe.to_csv("new_weather.csv")

In [18]:
state_weather_data = pd.read_csv("C:/Github/Group3_Project4/pete files/new_weather.csv")
state_weather_data = state_weather_data.set_index(['state'])
state_weather_data

,date,latitude,longitude,temperature_2m_max,temperature_2m_min
state,,,,,
Alabama,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.7991,27.679100
Alabama,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.6191,28.489100
Alabama,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.1891,29.569100
Alabama,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.5991,27.589100
Alabama,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.4291,35.869100
...,...,...,...,...,...
Wyoming,2023-12-27 00:00:00+00:00,42.776798,-107.440826,20.9327,6.802700
Wyoming,2023-12-28 00:00:00+00:00,42.776798,-107.440826,19.0427,6.982700
Wyoming,2023-12-29 00:00:00+00:00,42.776798,-107.440826,22.3727,12.202702


In [31]:
state_weather_data['date'] = pd.to_datetime(state_weather_data['date'], format = '%m/%d/%Y')

state_weather_data = state_weather_data.rename(columns = {'temperature_2m_max': 'temp_max', 'temperature_2m_min': 'temp_min'})

state_weather_data

,date,latitude,longitude,temp_max,temp_min
state,,,,,
Alabama,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.7991,27.679100
Alabama,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.6191,28.489100
Alabama,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.1891,29.569100
Alabama,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.5991,27.589100
Alabama,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.4291,35.869100
...,...,...,...,...,...
Wyoming,2023-12-27 00:00:00+00:00,42.776798,-107.440826,20.9327,6.802700
Wyoming,2023-12-28 00:00:00+00:00,42.776798,-107.440826,19.0427,6.982700
Wyoming,2023-12-29 00:00:00+00:00,42.776798,-107.440826,22.3727,12.202702
